# PLINK pre-QC in Bash


"Ride the PLINK wave." - Cristina

**Creating plink command to extract individuals for which eye colour data is available**

Files by the name "dataset_eyelist.txt" were created in the phenotype info documentation and are used as reference lists for keeping only the individuals who provided self-reported eye colour information. 

PLINK input filtering documentation can be found here: https://www.cog-genomics.org/plink/2.0/filter

Another useful resource for PLINK syntax can be found here: https://zzz.bwh.harvard.edu/plink/dataman.shtml


### Some notes for identifying the binary files

Refer to release notes under GRU to locate the following binary files:

**GENEVA**

    HapMap Controls: phg000070.v1.p1.GENEVA_Melanoma.genotype-calls-matrixfmt.MULTI.HapMap_controls.tar 

    Cases: phg000070.v1.p1.GENEVA_Melanoma.genotype-calls-matrixfmt.c1.GRU.tar 

    Controls: phg000094.v1.p1.GENEVA_Melanoma_CONTROLS.genotype-calls-matrixfmt.c1.GRU.tar 

**CIDR**

    HapMap Controls: phg000428.v1.CIDR_Melanoma_AuUK.genotype-calls-matrixfmt.MULTI.HapMap.tar

    Cases: phg000428.v1.CIDR_Melanoma_AuUK.genotype-calls-matrixfmt.c1.GRU-MDS.tar


**Duke**

    Human_1M: phg001278.v1.ImplicationsForDisease.genotype-calls-matrixfmt.Human_1M.c1.GRU-IRB-PUB.tar.gz

    Exome: phg001278.v1.ImplicationsForDisease.genotype-calls-matrixfmt.ExomeChip.c1.GRU-IRB-PUB.tar.gz


In [ ]:
# Frank CanPath data
./plink --bfile ATL_GENO --keep atl_eyelist.txt --make-bed --out ATL_eye
./plink --bfile ATP_GENO --keep atp_eyelist.txt --make-bed --out ATP_eye
./plink --bfile BCGP_GENO --keep bcgp_eyelist.txt --make-bed --out BCGP_eye
./plink --bfile OHS_GENO --keep ohs_eyelist.txt --make-bed --out OHS_eye

#Cristina CARTaGENE data (done qc and imputation already)
./plink --bfile CaG_GENO --keep cag_eyelist.txt --make-bed --out CaG_eye

#CIDR
./plink --bfile CIDR_Melanoma_AU_UK_Top_sample_level_c1 --keep CIDR_eyelist.txt --make-bed --out CIDR_CASES_eye

#DUKE
./plink --bfile dcc_deid_gwas_to_upload --keep DUKE_eyelist.txt --make-bed --out Duke_1M_eye

#GENEVA
./plink --bfile GENEVA_Melanoma_FORWARD_GRU --keep GENEVA_eyelist_v2.txt --make-bed --out GENEVA_CASES_eye
./plink --bfile GENEVA_Melanoma_FORWARD_CONTROLS_sample_level --keep GENEVA_eyelist_v2.txt --make-bed --out GENEVA_CONTROLS_eye

# PLINK QC in Bash

QC follows steps outlined in Marees et al. 2018.

Pre-imputation QC steps:
1. SNP QC, keep all SNPs with call rate > 0.95


2. Sample QC, keep all samples with
    
    a. Call rate > 0.95
    
    b. Sex check
    
    
3. SNP QC (again) – keep all SNPs with:

    a. Call rate > 0.98
    
    b. Missingness < 0.02
    
    c. MAF > 1%
    
    d. HWE p-values > 1e-6
    

**Missingness Check**

In [1]:
# OHS
./plink --bfile OHS_eye --missing --out OHS_miss

# BCGP
./plink --bfile BCGP_eye --missing --out BCGP_miss

# ATL
./plink --bfile ATL_eye --missing --out ATL_miss

# ATP
./plink --bfile ATP_eye --missing --out ATP_miss

# CIDR
./plink --bfile CIDR_CASES_eye --missing --out CIDR_miss

# DUKE
./plink --bfile DUKE_1M_eye --missing --out DUKE_miss

# GENEVA
./plink --bfile GENEVA_CASES_eye --missing --out GENEVA_CASES_miss
./plink --bfile GENEVA_CONTROLS_eye --missing --out GENEVA_CONTROLS_miss

# At this point, run either the following bash commands or run the missingness check manually from the QC_scripts.R file.
Rscript --no-save OHS_hist_miss.R
Rscript --no-save BCGP_hist_miss.R
Rscript --no-save ATL_hist_miss.R
Rscript --no-save ATP_hist_miss.R
Rscript --no-save CIDR_hist_miss.R
Rscript --no-save DUKE_hist_miss.R
Rscript --no-save GENEVA_hist_miss.R

bash: ./plink: No such file or directory
Fatal error: cannot open file 'hist_miss.R': No such file or directory


: 2

### STEP 1
**SNP QC, keep all SNPs with call rate > 0.95**

In [ ]:
# OHS
./plink --bfile OHS_eye --geno 0.05 --make-bed --out OHS_eye_s1

# BCGP
./plink --bfile BCGP_eye --geno 0.05 --make-bed --out BCGP_eye_s1

# ATL
./plink --bfile ATL_eye --geno 0.05 --make-bed --out ATL_eye_s1

# ATP
./plink --bfile ATP_eye --geno 0.05 --make-bed --out ATP_eye_s1

# CIDR
./plink --bfile CIDR_CASES_eye --geno 0.05 --make-bed --out CIDR_eye_s1

# DUKE
./plink --bfile DUKE_1M_eye --geno 0.05 --make-bed --out DUKE_eye_s1

# GENEVA
./plink --bfile GENEVA_CASES_eye --geno 0.05 --make-bed --out GENEVA_CASES_eye_s1
./plink --bfile GENEVA_CONTROLS_eye --geno 0.05 --make-bed --out GENEVA_CONTROLS_eye_s1

### STEP 2
**Sample QC, keep all samples with call rate > 0.98 and performing sex check**

In [2]:
# OHS
./plink --bfile OHS_eye_s1 --mind 0.02 --make-bed --out OHS_eye_s1_s2
./plink --bfile OHS_eye_s1_s2 --check-sex --out OHS_sex

# BCGP
./plink --bfile BCGP_eye_s1 --mind 0.02 --make-bed --out BCGP_eye_s1_s2
./plink --bfile BCGP_eye_s1_s2 --check-sex --out BCGP_sex

# ATL
./plink --bfile ATL_eye_s1 --mind 0.02 --make-bed --out ATL_eye_s1_s2
./plink --bfile ATL_eye_s1_s2 --check-sex --out ATL_sex

# ATP
./plink --bfile ATP_eye_s1 --mind 0.02 --make-bed --out ATP_eye_s1_s2
./plink --bfile ATP_eye_s1_s2 --check-sex --out ATP_sex

# CIDR
./plink --bfile CIDR_eye_s1 --mind 0.02 --make-bed --out CIDR_eye_s1_s2
./plink --bfile CIDR_eye_s1_s2 --check-sex --out CIDR_sex

# DUKE
./plink --bfile DUKE_eye_s1 --mind 0.02 --make-bed --out DUKE_eye_s1_s2
./plink --bfile DUKE_eye_s1_s2 --check-sex --out DUKE_sex

# GENEVA
./plink --bfile GENEVA_CASES_eye_s1 --mind 0.02 --make-bed --out GENEVA_CASES_eye_s1_s2
./plink --bfile GENEVA_CONTROLS_eye_s1 --mind 0.02 --make-bed --out GENEVA_CONTROLS_eye_s1_s2
./plink --bfile GENEVA_CASES_eye_s1_s2 --check-sex --out GENEVA_CASES_sex
./plink --bfile GENEVA_CONTROLS_eye_s1_s2 --check-sex --out GENEVA_CONTROLS_sex

# At this point, run either the following bash commands or run the sex check manually from the QC_scripts.R file.
Rscript --no-save OHS_sex_check.R
Rscript --no-save BCGP_sex_check.R
Rscript --no-save ATL_sex_check.R
Rscript --no-save ATP_sex_check.R
Rscript --no-save CIDR_sex_check.R
Rscript --no-save DUKE_sex_check.R
Rscript --no-save GENEVA_sex_check.R

bash: ./plink: No such file or directory
bash: ./plink: No such file or directory
bash: ./plink: No such file or directory
bash: ./plink: No such file or directory
bash: ./plink: No such file or directory
bash: ./plink: No such file or directory
Fatal error: cannot open file 'sex_check.R': No such file or directory


: 2

In [ ]:
## Two ways to deal with sex discrepancy.
    # If there are >~20 individuals with sex discrepancy, impute sex.
    # If there are <20 individuals, remove them.

# 1) Remove individuals with sex discrepancy.
grep "PROBLEM" CIDR_sex.sexcheck| awk '{print$1,$2}' > CIDR_sex_discrepancy.txt
grep "PROBLEM" DUKE_sex.sexcheck| awk '{print$1,$2}' > DUKE_sex_discrepancy.txt
grep "PROBLEM" GENEVA_CASES_sex.sexcheck| awk '{print$1,$2}' > GENEVA_CASES_sex_discrepancy.txt
grep "PROBLEM" GENEVA_CONTROLS_sex.sexcheck| awk '{print$1,$2}' > GENEVA_CONTROLS_sex_discrepancy.txt
# This command generates a list of individuals with the status "PROBLEM".

./plink --bfile CIDR_eye_s1_s2 --remove CIDR_sex_discrepancy.txt --make-bed --out CIDR_eye_s1_s2_sex
./plink --bfile DUKE_eye_s1_s2 --remove DUKE_sex_discrepancy.txt --make-bed --out DUKE_eye_s1_s2_sex
./plink --bfile GENEVA_CASES_eye_s1_s2 --remove GENEVA_CASES_sex_discrepancy.txt --make-bed --out GENEVA_CASES_eye_s1_s2_sex
./plink --bfile GENEVA_CONTROLS_eye_s1_s2 --remove GENEVA_CONTROLS_sex_discrepancy.txt --make-bed --out GENEVA_CONTROLS_eye_s1_s2_sex
# This command removes the list of individuals with the status "PROBLEM".

# OR

# 2) Impute sex - CanPath .fam files do not contain sex information, so we skipped sex imputation
./plink --bfile CIDR_eye_s1_s2 --impute-sex --make-bed --out CIDR_eye_s1_s2_sex
./plink --bfile DUKE_eye_s1_s2 --impute-sex --make-bed --out DUKE_eye_s1_s2_sex
./plink --bfile GENEVA_CASES_eye_s1_s2 --impute-sex --make-bed --out GENEVA_CASES_eye_s1_s2_sex
./plink --bfile GENEVA_CONTROLS_eye_s1_s2 --impute-sex --make-bed --out GENEVA_CONTROLS_eye_s1_s2_sex
# This imputes the sex based on the genotype information in the data set.

### STEP 3
**SNP QC again, keep all SNPS with call rate > 0.98, missingness < 0.02**

In [ ]:
# OHS
./plink --bfile OHS_eye_s1_s2 --geno 0.02 --make-bed --out OHS_eye_s1_s2_sex_s3

# BCGP
./plink --bfile BCGP_eye_s1_s2 --geno 0.02 --make-bed --out BCGP_eye_s1_s2_sex_s3

# ATL
./plink --bfile ATL_eye_s1_s2 --geno 0.02 --make-bed --out ATL_eye_s1_s2_sex_s3

# ATP
./plink --bfile ATP_eye_s1_s2 --geno 0.02 --make-bed --out ATP_eye_s1_s2_sex_s3

# CIDR
./plink --bfile CIDR_eye_s1_s2 --geno 0.02 --make-bed --out CIDR_eye_s1_s2_sex_s3

# DUKE
./plink --bfile DUKE_eye_s1_s2 --geno 0.02 --make-bed --out DUKE_eye_s1_s2_sex_s3

# GENEVA
./plink --bfile GENEVA_CASES_eye_s1_s2 --geno 0.02 --make-bed --out GENEVA_CASES_eye_s1_s2_sex_s3
./plink --bfile GENEVA_CONTROLS_eye_s1_s2 --geno 0.02 --make-bed --out GENEVA_CONTROLS_eye_s1_s2_sex_s3

**Keep all SNPs with MAF > 1%**

In [3]:
## This selects autosomal SNPs only (i.e., from chromosomes 1 to 22):

# OHS
awk '{ if ($1 >= 1 && $1 <= 22) print $2 }' OHS_eye_s1_s2_sex_s3.bim > OHS_snp_1_22.txt
./plink --bfile OHS_eye_s1_s2_sex_s3 --extract OHS_snp_1_22.txt --make-bed --out OHS_eye_s1_s2_sex_s3_auto

# BCGP
awk '{ if ($1 >= 1 && $1 <= 22) print $2 }' BCGP_eye_s1_s2_sex_s3.bim > BCGP_snp_1_22.txt
./plink --bfile BCGP_eye_s1_s2_sex_s3 --extract BCGP_snp_1_22.txt --make-bed --out BCGP_eye_s1_s2_sex_s3_auto

# ATL
awk '{ if ($1 >= 1 && $1 <= 22) print $2 }' ATL_eye_s1_s2_sex_s3.bim > ATL_snp_1_22.txt
./plink --bfile ATL_eye_s1_s2_sex_s3 --extract ATL_snp_1_22.txt --make-bed --out ATL_eye_s1_s2_sex_s3_auto

# ATP
awk '{ if ($1 >= 1 && $1 <= 22) print $2 }' ATP_eye_s1_s2_sex_s3.bim > ATP_snp_1_22.txt
./plink --bfile ATP_eye_s1_s2_sex_s3 --extract ATP_snp_1_22.txt --make-bed --out ATP_eye_s1_s2_sex_s3_auto

# CIDR
awk '{ if ($1 >= 1 && $1 <= 22) print $2 }' CIDR_eye_s1_s2_sex_s3.bim > CIDR_snp_1_22.txt
./plink --bfile CIDR_eye_s1_s2_sex_s3 --extract CIDR_snp_1_22.txt --make-bed --out CIDR_eye_s1_s2_sex_s3_auto

# DUKE
awk '{ if ($1 >= 1 && $1 <= 22) print $2 }' DUKE_eye_s1_s2_sex_s3.bim > DUKE_snp_1_22.txt
./plink --bfile DUKE_eye_s1_s2_sex_s3 --extract DUKE_snp_1_22.txt --make-bed --out DUKE_eye_s1_s2_sex_s3_auto

# GENEVA
awk '{ if ($1 >= 1 && $1 <= 22) print $2 }' GENEVA_CASES_eye_s1_s2_sex_s3.bim > GENEVA_CASES_snp_1_22.txt
./plink --bfile GENEVA_CASES_eye_s1_s2_sex_s3 --extract GENEVA_CASES_snp_1_22.txt --make-bed --out GENEVA_CASES_eye_s1_s2_sex_s3_auto
awk '{ if ($1 >= 1 && $1 <= 22) print $2 }' GENEVA_CONTROLS_eye_s1_s2_sex_s3.bim > GENEVA_CONTROLS_snp_1_22.txt
./plink --bfile GENEVA_CONTROLS_eye_s1_s2_sex_s3 --extract GENEVA_CONTROLS_snp_1_22.txt --make-bed --out GENEVA_CONTROLS_eye_s1_s2_sex_s3_auto

awk: can't open file OHS_eye_s1_s2_sex.bim
 source line number 1
bash: ./plink: No such file or directory
awk: can't open file OHS_eye_s1_s2_sex.bim
 source line number 1
bash: ./plink: No such file or directory


: 127

In [ ]:
## This generates a plot of the MAF distribution:

# OHS
./plink --bfile OHS_eye_s1_s2_sex_s3_auto --freq --out OHS_MAF_check

# BCGP
./plink --bfile BCGP_eye_s1_s2_sex_s3_auto --freq --out BCGP_MAF_check

# ATL
./plink --bfile ATL_eye_s1_s2_sex_s3_auto --freq --out ATL_MAF_check

# ATP
./plink --bfile ATP_eye_s1_s2_sex_s3_auto --freq --out ATP_MAF_check

# CIDR
./plink --bfile CIDR_eye_s1_s2_sex_s3_auto --freq --out CIDR_MAF_check

# DUKE
./plink --bfile DUKE_eye_s1_s2_sex_s3_auto --freq --out DUKE_MAF_check

# GENEVA
./plink --bfile GENEVA_CASES_eye_s1_s2_sex_s3_auto --freq --out GENEVA_CASES_MAF_check
./plink --bfile GENEVA_CONTROLS_eye_s1_s2_sex_s3_auto --freq --out GENEVA_CONTROLS_MAF_check

# At this point, run either the following bash commands or run the MAF check manually from the QC_scripts.R file.
Rscript --no-save OHS_MAF_check.R
Rscript --no-save BCGP_MAF_check.R
Rscript --no-save ATL_MAF_check.R
Rscript --no-save ATP_MAF_check.R
Rscript --no-save CIDR_MAF_check.R
Rscript --no-save DUKE_MAF_check.R
Rscript --no-save GENEVA_MAF_check.R

In [4]:
## Remove low MAF SNPs:

# OHS
./plink --bfile OHS_eye_s1_s2_sex_s3_auto --maf 0.01 --make-bed --out OHS_eye_s1_s2_sex_s3_auto_maf

# BCGP
./plink --bfile BCGP_eye_s1_s2_sex_s3_auto --maf 0.01 --make-bed --out BCGP_eye_s1_s2_sex_s3_auto_maf

# ATL
./plink --bfile ATL_eye_s1_s2_sex_s3_auto --maf 0.01 --make-bed --out ATL_eye_s1_s2_sex_s3_auto_maf

# ATP
./plink --bfile ATP_eye_s1_s2_sex_s3_auto --maf 0.01 --make-bed --out ATP_eye_s1_s2_sex_s3_auto_maf

# CIDR
./plink --bfile CIDR_eye_s1_s2_sex_s3_auto --maf 0.01 --make-bed --out CIDR_eye_s1_s2_sex_s3_auto_maf

# DUKE
./plink --bfile DUKE_eye_s1_s2_sex_s3_auto --maf 0.01 --make-bed --out DUKE_eye_s1_s2_sex_s3_auto_maf

# GENEVA
./plink --bfile GENEVA_CASES_eye_s1_s2_sex_s3_auto --maf 0.01 --make-bed --out GENEVA_CASES_eye_s1_s2_sex_s3_auto_maf
./plink --bfile GENEVA_CONTROLS_eye_s1_s2_sex_s3_auto --maf 0.01 --make-bed --out GENEVA_CONTROLS_eye_s1_s2_sex_s3_auto_maf

bash: ./plink: No such file or directory
bash: ./plink: No such file or directory
bash: ./plink: No such file or directory
bash: ./plink: No such file or directory
bash: ./plink: No such file or directory
bash: ./plink: No such file or directory
bash: ./plink: No such file or directory


: 127

**Keep all SNPs with HWEp > 1e-6**

In [ ]:
## Check the distribution of HWE p-values of all SNPs:

# OHS
./plink --bfile OHS_eye_s1_s2_sex_s3_auto_maf --hardy --out OHS_hwe

# BCGP
./plink --bfile BCGP_eye_s1_s2_sex_s3_auto_maf --hardy --out BCGP_hwe

# ATL
./plink --bfile ATL_eye_s1_s2_sex_s3_auto_maf --hardy --out ATL_hwe

# ATP
./plink --bfile ATP_eye_s1_s2_sex_s3_auto_maf --hardy --out ATP_hwe

# CIDR
./plink --bfile CIDR_eye_s1_s2_sex_s3_auto_maf --hardy --out CIDR_hwe

# DUKE
./plink --bfile DUKE_eye_s1_s2_sex_s3_auto_maf --hardy --out DUKE_hwe

# GENEVA
./plink --bfile GENEVA_CASES_eye_s1_s2_sex_s3_auto_maf --hardy --out GENEVA_CASES_hwe
./plink --bfile GENEVA_CONTROLS_eye_s1_s2_sex_s3_auto_maf --hardy --out GENEVA_CONTROLS_hwe

In [ ]:
## Selecting SNPs with HWE p-value below 0.00001, required for one of the two plot generated by the next Rscript, allows to zoom in on strongly deviating SNPs. 

# OHS
awk '{ if ($9 < 0.00001) print $0 }' OHS_hwe.hwe > OHS_hwe_zoomhwe.hwe

# BCGP
awk '{ if ($9 < 0.00001) print $0 }' BCGP_hwe.hwe > BCGP_hwe_zoomhwe.hwe

# ATL
awk '{ if ($9 < 0.00001) print $0 }' ATL_hwe.hwe > ATL_hwe_zoomhwe.hwe

# ATP
awk '{ if ($9 < 0.00001) print $0 }' ATP_hwe.hwe > ATP_hwe_zoomhwe.hwe

# CIDR
awk '{ if ($9 < 0.00001) print $0 }' CIDR_hwe.hwe > CIDR_hwe_zoomhwe.hwe

# DUKE
awk '{ if ($9 < 0.00001) print $0 }' DUKE_hwe.hwe > DUKE_hwe_zoomhwe.hwe

# GENEVA
awk '{ if ($9 < 0.00001) print $0 }' GENEVA_CASES_hwe.hwe > GENEVA_CASES_hwe_zoomhwe.hwe
awk '{ if ($9 < 0.00001) print $0 }' GENEVA_CONTROLS_hwe.hwe > GENEVA_CONTROLS_hwe_zoomhwe.hwe

# At this point, run either the following bash commands or run the HWE check manually from the QC_scripts.R file.
Rscript --no-save OHS_hwe.R
Rscript --no-save BCGP_hwe.R
Rscript --no-save ATL_hwe.R
Rscript --no-save ATP_hwe.R
Rscript --no-save CIDR_hwe.R
Rscript --no-save DUKE_hwe.R
Rscript --no-save GENEVA_hwe.R

In [ ]:
## Consider if you need to filter for both cases and controls.

    # if not (applies the same threshold to cases and controls): 

# OHS
./plink --bfile OHS_eye_s1_s2_sex_s3_auto_maf --hwe 1e-6 include-nonctrl --make-bed --out OHS_eye_s1_s2_sex_s3_auto_maf_hwe

# BCGP
./plink --bfile BCGP_eye_s1_s2_sex_s3_auto_maf --hwe 1e-6 include-nonctrl --make-bed --out BCGP_eye_s1_s2_sex_s3_auto_maf_hwe

# ATL
./plink --bfile ATL_eye_s1_s2_sex_s3_auto_maf --hwe 1e-6 include-nonctrl --make-bed --out ATL_eye_s1_s2_sex_s3_auto_maf_hwe

# ATP
./plink --bfile ATP_eye_s1_s2_sex_s3_auto_maf --hwe 1e-6 include-nonctrl --make-bed --out ATP_eye_s1_s2_sex_s3_auto_maf_hwe

# CIDR
./plink --bfile CIDR_eye_s1_s2_sex_s3_auto_maf --hwe 1e-6 include-nonctrl --make-bed --out CIDR_eye_s1_s2_sex_s3_auto_maf_hwe

# DUKE
./plink --bfile DUKE_eye_s1_s2_sex_s3_auto_maf --hwe 1e-6 include-nonctrl --make-bed --out DUKE_eye_s1_s2_sex_s3_auto_maf_hwe

# GENEVA
./plink --bfile GENEVA_CASES_eye_s1_s2_sex_s3_auto_maf --hwe 1e-6 include-nonctrl --make-bed --out GENEVA_CASES_eye_s1_s2_sex_s3_auto_maf_hwe
./plink --bfile GENEVA_CONTROLS_eye_s1_s2_sex_s3_auto_maf --hwe 1e-6 include-nonctrl --make-bed --out GENEVA_CONTROLS_eye_s1_s2_sex_s3_auto_maf_hwe

    # if yes (applies a less stringent threshold to cases (1e-10)):
    
# OHS
./plink --bfile OHS_eye_s1_s2_sex_s3_auto_maf --hwe 1e-10 --make-bed --out OHS_eye_s1_s2_sex_s3_auto_maf_hwe1
./plink --bfile OHS_eye_s1_s2_sex_s3_auto_maf_hwe1 --hwe 1e-6 include-nonctrl --make-bed --out OHS_eye_s1_s2_sex_s3_auto_maf_hwe2_clean1

# BCGP
./plink --bfile BCGP_eye_s1_s2_sex_s3_auto_maf --hwe 1e-10 --make-bed --out BCGP_eye_s1_s2_sex_s3_auto_maf_hwe1
./plink --bfile BCGP_eye_s1_s2_sex_s3_auto_maf_hwe1 --hwe 1e-6 include-nonctrl --make-bed --out BCGP_eye_s1_s2_sex_s3_auto_maf_hwe2_clean1

# ATL
./plink --bfile ATL_eye_s1_s2_sex_s3_auto_maf --hwe 1e-10 --make-bed --out ATL_eye_s1_s2_sex_s3_auto_maf_hwe1
./plink --bfile ATL_eye_s1_s2_sex_s3_auto_maf_hwe1 --hwe 1e-6 include-nonctrl --make-bed --out ATL_eye_s1_s2_sex_s3_auto_maf_hwe2_clean1

# ATP
./plink --bfile ATP_eye_s1_s2_sex_s3_auto_maf --hwe 1e-10 --make-bed --out ATP_eye_s1_s2_sex_s3_auto_maf_hwe1
./plink --bfile ATP_eye_s1_s2_sex_s3_auto_maf_hwe1 --hwe 1e-6 include-nonctrl --make-bed --out ATP_eye_s1_s2_sex_s3_auto_maf_hwe2_clean1

# CIDR
./plink --bfile CIDR_eye_s1_s2_sex_s3_auto_maf --hwe 1e-10 --make-bed --out CIDR_eye_s1_s2_sex_s3_auto_maf_hwe1
./plink --bfile CIDR_eye_s1_s2_sex_s3_auto_maf_hwe1 --hwe 1e-6 include-nonctrl --make-bed --out CIDR_eye_s1_s2_sex_s3_auto_maf_hwe2_clean1

# DUKE
./plink --bfile DUKE_eye_s1_s2_sex_s3_auto_maf --hwe 1e-10 --make-bed --out DUKE_eye_s1_s2_sex_s3_auto_maf_hwe1
./plink --bfile DUKE_eye_s1_s2_sex_s3_auto_maf_hwe1 --hwe 1e-6 include-nonctrl --make-bed --out DUKE_eye_s1_s2_sex_s3_auto_maf_hwe2_clean1

# GENEVA
./plink --bfile GENEVA_CASES_eye_s1_s2_sex_s3_auto_maf --hwe 1e-10 --make-bed --out GENEVA_CASES_eye_s1_s2_sex_s3_auto_maf_hwe1
./plink --bfile GENEVA_CASES_eye_s1_s2_sex_s3_auto_maf_hwe1 --hwe 1e-6 include-nonctrl --make-bed --out GENEVA_CASES_eye_s1_s2_sex_s3_auto_maf_hwe2_clean1
./plink --bfile GENEVA_CONTROLS_eye_s1_s2_sex_s3_auto_maf --hwe 1e-10 --make-bed --out GENEVA_CONTROLS_eye_s1_s2_sex_s3_auto_maf_hwe1
./plink --bfile GENEVA_CONTROLS_eye_s1_s2_sex_s3_auto_maf_hwe1 --hwe 1e-6 include-nonctrl --make-bed --out GENEVA_CONTROLS_eye_s1_s2_sex_s3_auto_maf_hwe2_clean1

### STEP 4
This is according to the Marees et al. tutorial:

**Removing Related Individuals**

In [ ]:
# It is essential to check datasets you analyse for cryptic relatedness.
# Assuming a random population sample, we exclude all individuals above pihat threshold of 0.2.
# The HapMap dataset is known to contain parent-offspring relations. 
# Check for relationships between individuals with a pihat > 0.2, then visualize these parent-offspring relations using z values. 

# OHS
./plink --bfile OHS_eye_s1_s2_sex_s3_auto_maf_hwe --genome --min 0.2 --out OHS_pihat_min0.2
awk '{ if ($8 >0.9) print $0 }' OHS_pihat_min0.2.genome > OHS_zoom_pihat.genome

# BCGP
./plink --bfile BCGP_eye_s1_s2_sex_s3_auto_maf_hwe --genome --min 0.2 --out BCGP_pihat_min0.2
awk '{ if ($8 >0.9) print $0 }' BCGP_pihat_min0.2.genome > BCGP_zoom_pihat.genome

# ATL
./plink --bfile ATL_eye_s1_s2_sex_s3_auto_maf_hwe --genome --min 0.2 --out ATL_pihat_min0.2
awk '{ if ($8 >0.9) print $0 }' ATL_pihat_min0.2.genome > ATL_zoom_pihat.genome

# ATP
./plink --bfile ATP_eye_s1_s2_sex_s3_auto_maf_hwe --genome --min 0.2 --out ATP_pihat_min0.2
awk '{ if ($8 >0.9) print $0 }' ATP_pihat_min0.2.genome > ATP_zoom_pihat.genome

# CIDR
./plink --bfile CIDR_eye_s1_s2_sex_s3_auto_maf_hwe --genome --min 0.2 --out CIDR_pihat_min0.2
awk '{ if ($8 >0.9) print $0 }' CIDR_pihat_min0.2.genome > CIDR_zoom_pihat.genome

# DUKE
./plink --bfile DUKE_eye_s1_s2_sex_s3_auto_maf_hwe --genome --min 0.2 --out DUKE_pihat_min0.2
awk '{ if ($8 >0.9) print $0 }' DUKE_pihat_min0.2.genome > DUKE_zoom_pihat.genome

# GENEVA
./plink --bfile GENEVA_CASES_eye_s1_s2_sex_s3_auto_maf_hwe --genome --min 0.2 --out GENEVA_CASES_pihat_min0.2
awk '{ if ($8 >0.9) print $0 }' GENEVA_CASES_pihat_min0.2.genome > GENEVA_CASES_zoom_pihat.genome
./plink --bfile GENEVA_CONTROLS_eye_s1_s2_sex_s3_auto_maf_hwe --genome --min 0.2 --out GENEVA_CONTROLS_pihat_min0.2
awk '{ if ($8 >0.9) print $0 }' GENEVA_CONTROLS_pihat_min0.2.genome > GENEVA_CONTROLS_zoom_pihat.genome


# At this point, run either the following bash commands or run the relatedness check manually from the QC_scripts.R file.
Rscript --no-save OHS_Relatedness.R
Rscript --no-save BCGP_Relatedness.R
Rscript --no-save ATL_Relatedness.R
Rscript --no-save ATP_Relatedness.R
Rscript --no-save CIDR_Relatedness.R
Rscript --no-save DUKE_Relatedness.R
Rscript --no-save GENEVA_Relatedness.R

In [ ]:
# Include only founders (individuals without parents in the dataset).
# Look again for individuals with a pihat > 0.2.
# For each pair of 'related' individuals with a pihat > 0.2, remove the individual with the lowest call rate. 

# OHS
./plink --bfile OHS_eye_s1_s2_sex_s3_auto_maf_hwe --filter-founders --make-bed --out OHS_eye_s1_s2_sex_s3_auto_maf_hwe_founders
./plink --bfile OHS_eye_s1_s2_sex_s3_auto_maf_hwe_founders --genome --min 0.2 --out OHS_pihat_min0.2_in_founders
./plink --bfile OHS_eye_s1_s2_sex_s3_auto_maf_hwe_founders --missing --out OHS_related_miss

# BCGP
./plink --bfile BCGP_eye_s1_s2_sex_s3_auto_maf_hwe --filter-founders --make-bed --out BCGP_eye_s1_s2_sex_s3_auto_maf_hwe_founders
./plink --bfile BCGP_eye_s1_s2_sex_s3_auto_maf_hwe_founders --genome --min 0.2 --out BCGP_pihat_min0.2_in_founders
./plink --bfile BCGP_eye_s1_s2_sex_s3_auto_maf_hwe_founders --missing --out BCGP_related_miss

# ATL
./plink --bfile ATL_eye_s1_s2_sex_s3_auto_maf_hwe --filter-founders --make-bed --out ATL_eye_s1_s2_sex_s3_auto_maf_hwe_founders
./plink --bfile ATL_eye_s1_s2_sex_s3_auto_maf_hwe_founders --genome --min 0.2 --out ATL_pihat_min0.2_in_founders
./plink --bfile ATL_eye_s1_s2_sex_s3_auto_maf_hwe_founders --missing --out ATL_related_miss

# ATP
./plink --bfile ATP_eye_s1_s2_sex_s3_auto_maf_hwe --filter-founders --make-bed --out ATP_eye_s1_s2_sex_s3_auto_maf_hwe_founders
./plink --bfile ATP_eye_s1_s2_sex_s3_auto_maf_hwe_founders --genome --min 0.2 --out ATP_pihat_min0.2_in_founders
./plink --bfile ATP_eye_s1_s2_sex_s3_auto_maf_hwe_founders --missing --out ATP_related_miss

# CIDR
./plink --bfile CIDR_eye_s1_s2_sex_s3_auto_maf_hwe --filter-founders --make-bed --out CIDR_eye_s1_s2_sex_s3_auto_maf_hwe_founders
./plink --bfile CIDR_eye_s1_s2_sex_s3_auto_maf_hwe_founders --genome --min 0.2 --out CIDR_pihat_min0.2_in_founders
./plink --bfile CIDR_eye_s1_s2_sex_s3_auto_maf_hwe_founders --missing --out CIDR_related_miss

# DUKE
./plink --bfile DUKE_eye_s1_s2_sex_s3_auto_maf_hwe --filter-founders --make-bed --out DUKE_eye_s1_s2_sex_s3_auto_maf_hwe_founders
./plink --bfile DUKE_eye_s1_s2_sex_s3_auto_maf_hwe_founders --genome --min 0.2 --out DUKE_pihat_min0.2_in_founders
./plink --bfile DUKE_eye_s1_s2_sex_s3_auto_maf_hwe_founders --missing --out DUKE_related_miss

# GENEVA
./plink --bfile GENEVA_CASES_eye_s1_s2_sex_s3_auto_maf_hwe --filter-founders --make-bed --out GENEVA_CASES_eye_s1_s2_sex_s3_auto_maf_hwe_founders
./plink --bfile GENEVA_CASES_eye_s1_s2_sex_s3_auto_maf_hwe_founders --genome --min 0.2 --out GENEVA_CASES_pihat_min0.2_in_founders
./plink --bfile GENEVA_CASES_eye_s1_s2_sex_s3_auto_maf_hwe_founders --missing --out GENEVA_CASES_related_miss
./plink --bfile GENEVA_CONTROLS_eye_s1_s2_sex_s3_auto_maf_hwe --filter-founders --make-bed --out GENEVA_CONTROLS_eye_s1_s2_sex_s3_auto_maf_hwe_founders
./plink --bfile GENEVA_CONTROLS_eye_s1_s2_sex_s3_auto_maf_hwe_founders --genome --min 0.2 --out GENEVA_CONTROLS_pihat_min0.2_in_founders
./plink --bfile GENEVA_CONTROLS_eye_s1_s2_sex_s3_auto_maf_hwe_founders --missing --out GENEVA_CONTROLS_related_miss


# Use a UNIX text editor (e.g., vi(m) ) to check which individual has the highest call rate in the 'related pair'. 

In [ ]:
# Generate a list of FID and IID of the individual(s) with a Pihat above 0.2, to check who had the lower call rate of the pair.
# In our dataset the individual 13291  NA07045 had the lower call rate.

# OHS
vi OHS_0.2_low_call_rate_pihat.txt
i 
13291  NA07045
# Press esc
:x
# Press enter

# BCGP
vi BCGP_0.2_low_call_rate_pihat.txt
i 
13291  NA07045
# Press esc
:x
# Press enter

# ATL
vi ATL_0.2_low_call_rate_pihat.txt
i 
13291  NA07045
# Press esc
:x
# Press enter

# ATP
vi ATP_0.2_low_call_rate_pihat.txt
i 
13291  NA07045
# Press esc
:x
# Press enter

# CIDR
vi CIDR_0.2_low_call_rate_pihat.txt
i 
13291  NA07045
# Press esc
:x
# Press enter

# DUKE
vi DUKE_0.2_low_call_rate_pihat.txt
i 
13291  NA07045
# Press esc
:x
# Press enter

# GENEVA
vi GENEVA_CASES_0.2_low_call_rate_pihat.txt
i 
13291  NA07045
# Press esc
:x
# Press enter

vi GENEVA_CONTROLS_0.2_low_call_rate_pihat.txt
i 
13291  NA07045
# Press esc
:x
# Press enter


In [ ]:
# Delete the individuals with the lowest call rate in 'related' pairs with a pihat > 0.2 
# 0 non-founders in CanPath, CIDR, DUKE data, so we don't remove anything

# OHS
./plink --bfile OHS_eye_s1_s2_sex_s3_auto_maf_hwe_founders --make-bed --out OHS_eye_s1_s2_sex_s3_s4

# BCGP
./plink --bfile BCGP_eye_s1_s2_sex_s3_auto_maf_hwe_founders --make-bed --out BCGP_eye_s1_s2_sex_s3_s4

# ATL
./plink --bfile ATL_eye_s1_s2_sex_s3_auto_maf_hwe_founders --make-bed --out ATL_eye_s1_s2_sex_s3_s4

# ATP
./plink --bfile ATP_eye_s1_s2_sex_s3_auto_maf_hwe_founders --make-bed --out ATP_eye_s1_s2_sex_s3_s4

# CIDR
./plink --bfile CIDR_eye_s1_s2_sex_s3_auto_maf_hwe_founders --make-bed --out CIDR_eye_s1_s2_sex_s3_s4_CLEAN

# DUKE
./plink --bfile DUKE_eye_s1_s2_sex_s3_auto_maf_hwe_founders --make-bed --out DUKE_eye_s1_s2_sex_s3_s4_CLEAN

# GENEVA
./plink --bfile GENEVA_CASES_eye_s1_s2_sex_s3_auto_maf_hwe_founders --make-bed --out GENEVA_CASES_eye_s1_s2_sex_s3_s4_CLEAN
./plink --bfile GENEVA_CONTROLS_eye_s1_s2_sex_s3_auto_maf_hwe_founders --make-bed --out GENEVA_CONTROLS_eye_s1_s2_sex_s3_s4_CLEAN


### STEP 5
This is according to the Marees et al. tutorial:

**Pruning**

In [5]:
## Pruning done only on CanPath datasets. 

    # Therefore, to generate a list of non-(highly)correlated SNPs, we exclude high inversion regions (inversion.txt [High LD regions]) and prune the SNPs using the command --indep-pairwise.
    # The parameters "50 5 0.2" stand for:  window size, number of SNPs to shift the window at each step, and multiple correlation coefficient for a SNP being regressed on all other SNPs simultaneously.

# Don't delete the file indepSNP.prune.in

# OHS
./plink --bfile OHS_eye_s1_s2_sex_s3_s4 --indep-pairwise 50 5 0.2 --out OHS_indepSNP
./plink --bfile OHS_eye_s1_s2_sex_s3_s4 --extract OHS_indepSNP.prune.in --make-bed --out OHS_eye_s1_s2_sex_s3_s4_pruned

# BCGP
./plink --bfile BCGP_eye_s1_s2_sex_s3_s4 --indep-pairwise 50 5 0.2 --out BCGP_indepSNP
./plink --bfile BCGP_eye_s1_s2_sex_s3_s4 --extract BCGP_indepSNP.prune.in --make-bed --out BCGP_eye_s1_s2_sex_s3_s4_pruned

# ATL
./plink --bfile ATL_eye_s1_s2_sex_s3_s4 --indep-pairwise 50 5 0.2 --out ATL_indepSNP
./plink --bfile ATL_eye_s1_s2_sex_s3_s4 --extract ATL_indepSNP.prune.in --make-bed --out ATL_eye_s1_s2_sex_s3_s4_pruned

# ATP
./plink --bfile ATP_eye_s1_s2_sex_s3_s4 --indep-pairwise 50 5 0.2 --out ATP_indepSNP
./plink --bfile ATP_eye_s1_s2_sex_s3_s4 --extract ATP_indepSNP.prune.in --make-bed --out ATP_eye_s1_s2_sex_s3_s4_pruned

# done for now, then go to pca

=======================



bash: ./plink: No such file or directory
bash: ./plink: No such file or directory
bash: ./plink: No such file or directory
bash: =======================: command not found
Fatal error: cannot open file 'check_heterozygosity_rate.R': No such file or directory
bash: syntax error near unexpected token `('
bash: syntax error near unexpected token `"heterozygosity.pdf"'
bash: syntax error near unexpected token `('
bash: syntax error near unexpected token `het$HET_RATE,'
bash: syntax error near unexpected token `Rscript'
bash: syntax error near unexpected token `('
bash: syntax error near unexpected token `('
bash: syntax error near unexpected token `('
bash: syntax error near unexpected token `('
bash: syntax error near unexpected token `het_fail,'
sed: fail-het-qc.txt: No such file or directory
bash: ./plink: No such file or directory


: 127

### Now that you're done, make sure to keep the following files:
1. Binary file output (OHS_eye_s1_s2_sex_s3_s4_pruned or CIDR_eye_s1_s2_sex_s3_s4_CLEAN)
2. indepSNP.prune.in


# Preparing QC Output for Imputation

**Create script files for steps 1 and 2 and name them something like:**

1. "vcf_convert.sh"
2. "splitchr_index.sh"

**1. Convert binary files to vcf using PLINK**

Using a loop proved a little difficult so I did each dataset manually.

OHS, BCGP, ATL, ATP successfully imputed using TopMed.

The dbGaP datasets suffered from strand flip and clumping issues, so there is a "snps-only" tag to exclude any multi-character alleles. Frank provided some files + code to fix strand flip issues, this is in the "Imputation + dbGaP Imputation issues" file.

In [ ]:
#!/bin/bash

#SBATCH --account=def-wendtfra
#SBATCH --time=15:00:00
#SBATCH --job-name=new-convert-vcf
#SBATCH --output=new-convert-vcf.out
#SBATCH --error=new-convert-vcf.err
#SBATCH --mail-user=brendan.newton@mail.utoronto.ca
#SBATCH --mail-type=ALL
#SBATCH --mem=10G

module load StdEnv/2020 plink/1.9b_6.21-x86_64

# Removing REF/ALT alleles that are not A,T,C,G
#Convert PLINK binary files to VCF format directly
plink --bfile /home/newtonb1/scratch/PLINK/plink_mac_20230116/OHS_eye_s1_s2_sex_s3_s4_pruned --snps-only 'just-acgt'--recode vcf --out OHS_pruned.vcf
plink --bfile /home/newtonb1/scratch/PLINK/plink_mac_20230116/BCGP_eye_s1_s2_sex_s3_s4_pruned --snps-only 'just-acgt' --recode vcf --out BCGP_pruned.vcf
plink --bfile /home/newtonb1/scratch/PLINK/plink_mac_20230116/ATL_eye_s1_s2_sex_s3_s4_pruned --snps-only 'just-acgt' --recode vcf --out ATL_pruned.vcf
plink --bfile /home/newtonb1/scratch/PLINK/plink_mac_20230116/ATP_eye_s1_s2_sex_s3_s4_pruned --snps-only 'just-acgt' --recode vcf --out ATP_pruned.vcf
plink --bfile /home/newtonb1/scratch/PLINK/plink_mac_20230116/DUKE_eye_s1_s2_sex_s3_s4_CLEAN --snps-only 'just-acgt' --recode vcf --out DUKE_pruned.vcf
plink --bfile /home/newtonb1/scratch/PLINK/plink_mac_20230116/CIDR_eye_s1_s2_sex_s3_s4_CLEAN --snps-only 'just-acgt' --recode vcf --out CIDR_pruned.vcf
plink --bfile /home/newtonb1/scratch/PLINK/plink_mac_20230116/GENEVA_CASES_eye_s1_s2_sex_s3_s4_CLEAN --snps-only 'just-acgt' --recode vcf --out GENEVA_CASES_pruned.vcf
plink --bfile /home/newtonb1/scratch/PLINK/plink_mac_20230116/GENEVA_CONTROLS_eye_s1_s2_sex_s3_s4_CLEAN --snps-only 'just-acgt' --recode vcf --out GENEVA_CONTROLS_pruned.vcf

**2. Separate by chromosome and index**

This step is required because the input for TopMed must be split by chromosome

In [ ]:
#!/bin/bash

#SBATCH --account=def-wendtfra
#SBATCH --time=15:00:00
#SBATCH --job-name=new-test-splitchr
#SBATCH --output=new-test-splitchr.out
#SBATCH --error=new-test-splitchr.err
#SBATCH --mail-user=brendan.newton@mail.utoronto.ca
#SBATCH --mail-type=ALL
#SBATCH --mem=10G

module load StdEnv/2020 gcc/9.3.0 samtools/1.17  bcftools/1.13

#Define chromosome numbers and dataset names
chr=( {1..22} )
datasets=("OHS" "BCGP" "ATL" "ATP" "DUKE" "CIDR" "GENEVA_CASES" "GENEVA_CONTROLS")

# Loop over each chromosome and dataset
for dataset in "${datasets[@]}"
do
        # Input VCF file name
        input_vcf="${dataset}_pruned.vcf"
        compressed_vcf="${dataset}_pruned.vcf.gz"
        # Index before splitting by chromosome
        bgzip -c ${input_vcf} > ${compressed_vcf}
        bcftools index ${compressed_vcf}

        for chr_num in "${chr[@]}"
        do
                # Output VCF file names
                output_vcf="${dataset}-chr${chr_num}.vcf.gz"
                # Extract variants for current chromosome
                bcftools view -r ${chr_num} ${compressed_vcf} | bgzip -c > ${output_vcf}
                # Index again
                bcftools index ${output_vcf}
        done
done

At this point, we run the java code that Frank sent, in the "Imputation + dbGaP Imputation issues" file. 

In [ ]:
# Move Frank's files to Cedar with
scp -r /Users/brendannewton/Desktop/WENDT_LAB/FSC485/FRANK_IMPUTATION newtonb1@cedar.computecanada.ca:/home/newtonb1/scratch/NEW_INDEX

In [ ]:
#!/bin/bash
#SBATCH --job-name=strandflip-fix
#SBATCH --account=def-wendtfra
#SBATCH --mem=1000
#SBATCH --time=10:00:00
#SBATCH --output=strandflip-fix.out
#SBATCH --error=strandflip-fix.err
#SBATCH --mail-user=brendan.newton@mail.utoronto.ca
#SBATCH --mail-type=ALL

#####DOWNLOADS#####
#go to this website and download the JAVA executable at the bottom of the page: https://faculty.washington.edu/browning/conform-gt.html#use
#also click on "human reference panel", click on "b37.vcf", download each of the .vcf.gz files for the autosomes. the .tbi files are not needed.
#move everything into compute canada scratch directory

#####THINGS TO CHANGE ABOUT YOUR INPUT DATA#####
#this script will not accept anything in the dbGAP .vcf files that has a REF or ALT allele other than A, C, T, or G so you will need to remove those.
#PLINK and VCFtools should both have something like a 'snps-only' flag that can help you do this pretty easily.

#####SCRIPTS#####
#Keep the .sh header of this file. You might need to modify the wall time but everything else should be ok to keep as is.
module load plink/1.9b_6.21-x86_64
module load vcftools/0.1.16
module load java/17.0.2

java -Xmx8g -jar /home/wendtfra/conform-gt.24May16.cee.jar ref=chr22.1kg.phase3.v5a.vcf gt=DUKE_chr22.vcf.gz chrom=22 out=new.chr22 excludesamples=non.eur.excl

chr=( {1..22} )
datasets=("OHS" "BCGP" "ATL" "ATP" "DUKE" "CIDR" "GENEVA_CASES" "GENEVA_CONTROLS")

for dataset in "${datasets[@]}"
do
    for chr_num in "${chr[@]}"
    do
        java -Xmx8g -jar /home/newtonb1/scratch/NEW_INDEX/conform-gt.24May16.cee.jar ref=chr${chr}.1kg.phase3.v5a.vcf.gz gt=${dataset}_chr${chr}.vcf.gz chrom=22 out=${dataset}_new.chr${chr} excludesamples=non.eur.excl
    done
done

OHS, BCGP, ATL, ATP should successfully impute without having to remove SNPs.

The java script will compare the alleles with the reference sample and, if different, will switch A1 and A2 columns.

## Moving File Directory to Cedar

**Use the following command in your local home directory**

In [ ]:
scp -r /Users/brendannewton/Desktop/WENDT_LAB/FSC485/UNIMPUTED newtonb1@cedar.computecanada.ca:/home/newtonb1/scratch

Alternatively, you can use MobaXTerm or CyberDuck to move/create/delete folders and files easily

## Post-Imputation QC + Merge

**dbGap Imputation issues not fixed, move to use CanPath data. These commands were not used (provided by Cristina) but would have been to perform steps missing between Imputation and PCA.**

Can also use IMMerge for the merging step: https://academic.oup.com/bioinformatics/article/39/1/btac750/6839927

In [ ]:
### Extract list of variants to reduce missingness in merged file:

#!/bin/bash

#SBATCH --account=def-eparra
#SBATCH --time=24:00:00
#SBATCH --job-name=extract-variant-list
#SBATCH --output=extract-variant-list.out
#SBATCH --mail-user=c.abbatangelo@mail.utoronto.ca
#SBATCH --mail-type=ALL
#SBATCH --mem=20G

module load StdEnv/2020 htslib/1.17

# chr${i}.Axiom.R2-clean.recode.vcf.gz
# chr${i}.CartaGene-clean.recode.vcf.gz

# Loop through the files using the naming convention chr1.vcf.gz, chr2.vcf.gz
for i in {1..22}; do
    vcf_file="chr${i}.Axiom.R2-clean.recode.vcf.gz"
    output_txt="variant_list_chr${i}.txt"

    # Use grep to extract variant lines and append to the output file
    zcat "$vcf_file" | grep -v "^#" | cut -f 1-5 >> "$output_txt"

    echo "Variant list generation for chr${i} complete."
done

echo "All variant lists generated."


In [ ]:
#!/bin/bash

#SBATCH --account=def-eparra
#SBATCH --time=24:00:00
#SBATCH --job-name=extract-variant-list-chr1
#SBATCH --output=extract-variant-list-chr1.out
#SBATCH --error=#SBATCH --output=extract-variant-list-chr1.err
#SBATCH --mail-user=c.abbatangelo@mail.utoronto.ca
#SBATCH --mail-type=ALL
#SBATCH --mem=20G

module load StdEnv/2020 htslib/1.17 tabix/0.2.6

# Copy header lines from the original VCF file to the output file
zcat chr1.CartaGene-clean.recode.vcf.gz | grep "^#" > chr1.CartaGene-clean-varlist.vcf

# Use grep to filter variants from dataset 2 using the variant list
zcat chr1.CartaGene-clean.recode.vcf.gz | grep -Ff variant_list_chr1.txt >> chr1.CartaGene-clean-varlist.vcf

# Compress the output VCF file
bgzip chr1.CartaGene-clean-varlist.vcf

# Index the compressed VCF file using tabix
tabix -p vcf chr1.CartaGene-clean-varlist.vcf.gz

echo "Variant extraction for chr1 from dataset 2 complete."

## Turn this into a loop or array to do multiple chromosomes at one


In [ ]:
### Merge:

#!/bin/bash
#SBATCH --account=def-eparra
#SBATCH --time=15:00:00
#SBATCH --job-name=merge-chr1  
#SBATCH --output=merge-chr1.out
#SBATCH --error=merge-chr1.err
#SBATCH --mail-user=c.abbatangelo@mail.utoronto.ca
#SBATCH --mail-type=ALL
#SBATCH --mem=20G

module load StdEnv/2020  gcc/9.3.0 bcftools/1.16
module load tabix/0.2.6

# Command suggested by G. Debortoli bcftools concat file1.vcf file2.vcf -Oz > all_chr.vcf.gz

#gzip -d chr22.CartaGene-clean-varlist.vcf.gz

#bgzip chr22.CartaGene-clean-varlist.vcf

    vcf_file1="chr1.Axiom.R2-clean.recode.vcf.gz"
    vcf_file2="chr1.CartaGene-clean-varlist.vcf.gz"
    output_file="chr1-merged-AllCanPath.vcf.gz"

    # Merge VCF files using vcftools
    bcftools merge "$vcf_file1" "$vcf_file2" -Oz > "$output_file"

    # Index the merged VCF file
    #tabix -p vcf chr22-merged-AllCanPath.vcf.gz

    echo "Merged chr1 complete."


In [ ]:
### Final QC:

#!/bin/bash

#SBATCH --account=def-eparra
#SBATCH --time=48:00:00
#SBATCH --job-name=final-autosome-qc
#SBATCH --array=1-22
#SBATCH --output=final-autosome-qc.out
#SBATCH --error=final-autosome-qc.err
#SBATCH --mail-user=c.abbatangelo@mail.utoronto.ca
#SBATCH --mail-type=ALL
#SBATCH --mem=100G

module load StdEnv/2020 gcc/9.3.0 bcftools/1.13 vcftools/0.1.16 tabix/0.2.6

# Input VCF file
input_vcf="chr${SLURM_ARRAY_TASK_ID}-merged-AllCanPath.vcf.gz"

# Output filtered and compressed VCF file
output_filtered="chr${SLURM_ARRAY_TASK_ID}_merged_AllCanPath_clean.vcf.gz"

# Filtering using vcftools
vcftools --gzvcf "$input_vcf" --maf 0.005 --hwe 1e-6 --max-missing 0.98 --recode --recode-INFO-all --stdout | bgzip -c > "$output_filtered"

# Indexing the filtered and compressed VCF file using tabix
tabix -p vcf "$output_filtered"


## PCA

PLINK --pca flag was ported from GCTA. Generates plink.eigenvec and plink.eigenval files.

These can be run locally.

In [ ]:
./plink --pca

# Relevant PLINK input documentation 

### --keep info

In [ ]:
keep <filename(s)...>
--remove <filename(s)...>
--keep-fam <filename(s)...>
--remove-fam <filename(s)...>

--keep accepts one or more space/tab-delimited text files with sample IDs, and removes all unlisted samples from the current analysis; --remove does the same for all listed samples. Similarly, --keep-fam and --remove-fam accept text files with family IDs in the first column, and keep or remove entire families.

--keep/--remove now support a wider variety of sample ID file formats:

If the first line starts with '#FID' or '#IID', it will be treated as a header line. As long as the first columns are "#FID IID", "#FID IID SID", "#IID", or "#IID SID", PLINK 2 will do the right thing. (Note that when FID is undefined, it is treated as '0'.)
If there is no header line, one-column lines are treated as IIDs, and multicolumn lines are treated the same way as in PLINK 1.x (first two columns assumed to be FID/IID).

**Phenotype/covariate-based**

--keep-if removes all samples which don't satisfy a comparison predicate on a phenotype or covariate, while --remove-if does the reverse.

In [ ]:
--keep-if <phenotype/covariate name> <operator> <value>
--remove-if <phenotype/covariate name> <operator> <value>

Syntax and treatment of missing values is the same as for --extract-if-info.
For binary phenotypes, either '2' or 'case' (any capitalization) can be used to refer to cases, and either '1', 'ctrl', or 'control' can be used to refer to controls.

In [ ]:
--require-pheno [phenotype name(s)...]
--require-covar [covariate name(s)...]

When parameters are provided, --require-pheno removes samples missing any of the named phenotypes; otherwise, it removes samples missing any loaded phenotype. --require-covar does the same things for covariates.

### Missing genotype rates

In [ ]:
--geno [maximum per-variant]
--mind [maximum per-sample]
--oblig-missing <variant x block file> <block definition file>

--geno filters out all variants with missing call rates exceeding the provided value (default 0.1) to be removed, while --mind does the same for samples.

--oblig-missing lets you specify blocks of missing genotype calls for --geno and --mind to ignore. The first file should be a text file with variant IDs in the first column and block names in the second, while the second file should be in .clst format. See the PLINK 1.07 documentation for examples. (--oblig-clusters is a deprecated way to specify --oblig-missing's second parameter.)

If any genotype calls in a block are not actually missing, PLINK now reports an error; use --zero-cluster if you want to force those calls to missing instead.

### MAF

In [ ]:
--maf [minimum freq]
--max-maf <maximum freq>
--mac <minimum count>
  (alias: --min-ac)
--max-mac <maximum count>
  (alias: --max-ac)

--maf filters out all variants with minor allele frequency below the provided threshold (default 0.01), while --max-maf imposes an upper MAF bound. Similarly, --mac and --max-mac impose lower and upper minor allele count bounds, respectively.

Only founders are normally considered by these filters; use --nonfounders to change this.

--maf-succ

--maf-succ causes primary minor allele frequencies to be estimated via the "rule of succession" employed by EIGENSOFT. I.e.,

   qhat := (1 + <observed minor allele count>) / (2 + <total observations>)

instead of the usual

   qhat := <observed minor allele count> / <total observations>.

This flag does not affect stratified MAF computations.

### HWE

In [ ]:
--hwe <p-value> ['midp'] ['include-nonctrl']

--hwe filters out all variants which have Hardy-Weinberg equilibrium exact test p-value below the provided threshold. We recommend setting a low threshold—serious genotyping errors often yield extreme p-values like 1e-50 which are detected by any reasonable configuration of this test, while genuine SNP-trait associations can be expected to deviate slightly from Hardy-Weinberg equilibrium (so it's dangerous to choose a threshold that filters out too many variants). This HWE p-value calculator may be helpful.

--hwe's 'midp' modifier applies the mid-p adjustment described in Graffelman J, Moreno V (2013) The mid p-value in exact tests for Hardy-Weinberg equilibrium. The mid-p adjustment tends to bring the null rejection rate in line with the nominal p-value, and also reduces the filter's tendency to favor retention of variants with missing data. We recommend its use.

Because of the missing data issue, you should not apply a single p-value threshold across a batch of variants with highly variable missing call rates. A warning is now given whenever observation counts vary by more than 10%.

Only founders are considered by this test; use --nonfounders to change this. Also, with case/control data, cases and missing phenotypes are normally ignored; override this with 'include-nonctrl'.

### --freq

In [ ]:
--freq [{counts | case-control}] ['gz']
--freqx ['gz']
  (alias: --frqx)

By itself, --freq writes a minor allele frequency report to plink.frq. If you add the 'counts' modifier, an allele count report is written to plink.frq.count instead. Alternatively, you can use --freq with --within/--family to write a cluster-stratified frequency report to plink.frq.strat, or use the 'case-control' modifier to write a case/control phenotype-stratified report to plink.frq.cc.

--freqx writes a more informative genotype count report to plink.frqx.

For both flags, gzipped output can be requested with the 'gz' modifier.

Nonfounders are normally excluded from these counts/frequencies; use --nonfounders to change this.

All of these reports (except for --freq + --within/--family) are valid input for --read-freq; --freqx is the most powerful when used in that capacity, since it preserves deviation from Hardy-Weinberg equilibrium.

### --extract and --exclude

In [ ]:
--extract ['range'] <filename>
--exclude ['range'] <filename>

--extract normally accepts a text file with a list of variant IDs (usually one per line, but it's okay for them to just be separated by spaces), and removes all unlisted variants from the current analysis.

With the 'range' modifier, the input file should be in set range format instead.

--exclude does the same for all listed variants. Note that this is slightly different from PLINK 1.07's behavior when the main input fileset contains duplicate variant IDs: PLINK 1.9 removes all matches, while PLINK 1.07 just removes one of the matching variants. If your intention is to resolve duplicates, you should now use --bmerge instead of --exclude.

### --range

In [ ]:
plink --file data --extract myrange.txt --range

--range is a modifier for --extract and --exclude. If the --range flag is added, then instead of a list of SNPs, PLINK will expect a list of chromosomal ranges to be given instead, one per line.

All SNPs within that range will then be excluded or extracted. The format of myrange.txt should be, one range per line, whitespace-separated:

     CHR     Chromosome code (1-22, X, Y, XY, MT, 0)     
     BP1     Start of range, physical position in base units     
     BP2     End of range, as above     
     LABEL   Name of range/gene
     
For example,

     2 30000000 35000000  R1
     2 60000000 62000000  R2     
     X 10000000 20000000  R3
     
would extract/exclude all SNPs in these three regions (5Mb and 2Mb on chromosome 2 and 10Mb on chromosome X).

### --snps-only

In [ ]:
--snps-only ['just-acgt']

--snps-only excludes all variants with one or more multi-character allele codes. With 'just-acgt', variants with single-character allele codes outside of {'A', 'C', 'G', 'T', 'a', 'c', 'g', 't', } are also excluded.

### --pca

In [ ]:
--pca [count] ['header'] ['tabs'] ['var-wts']
--pca-cluster-names <name(s)...>
--pca-clusters <filename>

By default, --pca extracts the top 20 principal components of the variance-standardized relationship matrix; you can change the number by passing a numeric parameter. Eigenvectors are written to plink.eigenvec, and top eigenvalues are written to plink.eigenval. The 'header' modifier adds a header line to the .eigenvec file(s), and the 'tabs' modifier makes the .eigenvec file(s) tab- instead of space-delimited. You can request variant weights with the 'var-wts' modifier, and dump the matrix by using --pca in combination with --make-rel/--make-grm-gz/--make-grm-bin.

For other instructions see: https://www.cog-genomics.org/plink/1.9/strat#pca